In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
schema = StructType(fields=[StructField('rooms_nmb', IntegerType(), False),
                            StructField('sqm', IntegerType(), False),
                            StructField('floor_nmb', StringType(), False),                                
                            StructField('district', StringType(), False),
                            StructField('address', StringType(), True),
                            StructField('rent_price', IntegerType(), False),
                            StructField('owner_type', StringType(), True),
                            StructField('publication_date', StringType(), False),
                            StructField('url', StringType(), False)])

In [0]:
krisha_df = spark.read\
                 .format('csv')\
                 .option('delimiter', ';')\
                 .schema(schema)\
                 .load('/mnt/krisha042023/almaty-raw/*')

In [0]:
final_df = krisha_df\
           .withColumn('floor_nmb', split(col('floor_nmb'), '/')[0])\
           .withColumn('id', split(col('url'), '/')[3])\
           .withColumn('batch_timestamp', current_timestamp())\
           .select('id', 'district', 'address', 'rooms_nmb', 'sqm', 
                   'floor_nmb', 'rent_price', 'owner_type', 
                   'url', 'publication_date', 'batch_timestamp')

In [0]:
final_df.write\
        .partitionBy('district')\
        .mode('overwrite')\
        .format('parquet')\
        .save('/mnt/krisha042023/almaty-processed/parquet')

In [0]:
# Separate dataframe in csv format for dashboards
final_df.write\
        .mode('overwrite')\
        .option('header', 'true')\
        .format('csv')\
        .save('/mnt/krisha042023/almaty-processed/for_power_bi/')